<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
موضوع‌بندی
</font>
</h1>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مقدمه و صورت مسئله
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
به یک تمرین صنعتی از  کاربرد یادگیری ماشین در پردازش زبان طبیعی (NLP) خوش آمدید.  در این تمرین داده‌های واقعی وب فارسی که توسط پلتفرم <a href="https://www.yektanet.com/">یکتانت</a> پالایش و جمع‌آوری شده در اختیار ما قرار گرفته است. هدف تمرین؛ ساخت یک مدل یادگیری ماشین است که با توجه متن‌های موجود در یک پیوند (Link) نظیر <i>عنوان</i>، <i>توضیحات</i>، <i>محتوای متنی [کامل]</i> و غیره بتواند دسته‌ی موضوعی آن سند را پیش‌بینی کند. به‌عنوان مثال اگر پیوندی از یک سایت خبری با عنوان «<i>کیهان کلهر جایزه مرد سال موسیقی جهان را دریافت کرد</i>» داشته باشیم، مدل ما باید پیش‌بینی کند که این مطلب مرتبط با موضوع «موسیقی» است. البته در این مثال ما تنها از ویژگی <i>عنوان</i> یاد کردیم، در حالی‌که می‌توان از متن <i>توضیحات</i> یا <i>محتوای متنی</i> هم کمک گرفت. 

</br>
توجه داشته باشید برای آن‌که بتوانید از الگوریتم‌های معرفی‌شده در کالج جهت کار روی داده‌های متنی استفاده کنید نیاز است حداقل با یکی از روش‌های تعبیه‌سازی (Embedding) آشنا شده باشید تا بتوانید هر متن را به یک بردار ویژگی عددی تبدیل کنید.

</font>
</p>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
وارد کردن کتابخانه‌های مورد نیاز
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    ابتدا کتابخانه‌های مورد نیازتان را وارد کنید.
</font>
</p>

In [234]:
import numpy as np
import pandas as pd
import hazm as hz
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import emoji
from xgboost import XGBClassifier

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه‌داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    هر نمونه از این مجموعه‌داده با ویژگی‌هایی که در جدول زیر شرح داده شده همراه است. ستون <code>category</code> متغیر هدف مسئله است که موضوع محتوا را نشان می‌دهد. 
</font>
</p>
<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|<code>category</code>| موضوع (متغیر هدف) |
|<code>description</code>| توضیحات |
|<code>text_content</code>| محتوای متنی |
|<code>title</code>| عنوان |
|<code>h1</code>| محتوای تگ <code>h1</code> صفحه |
|<code>h2</code>|محتوای تگ <code>h2</code> صفحه  |
|<code>url</code>| آدرس پیوند|
|<code>domain</code>|دامنه‌ی وب‌سایت |
|<code>id</code>|آیدی پیوند|

</font>
</div>
</center>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خواندن مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    در ابتدا نیاز است فایل‌های مجموعه‌داده را بخوانید. نمونه‌های آموزشی در فایل <code>yektanet_train.csv</code> و نمونه‌های آزمون که باید موضوع آن‌ها را پیش‌بینی کنید در فایل <code>yektanet_test.csv</code> ذخیره شده‌اند. اگر لازم دانستید می‌توانید به دلخواه خود بخشی از مجموعه‌ی آموزشی را به عنوان مجموعه‌ی اعتبارسنجی نیز جدا کنید.
</font>
</p>

In [235]:
train = pd.read_csv('../data/yektanet_train.csv')
train.head()

,category,description,text_content,title,h1,h2,url,domain,id
0,کتاب و ادبیات,"از شوبنده ها: جستجو معنی ""از شوبنده ها"" در فره...",معنی از شوبنده ها | جدول یاب از شوبنده ها 381,معنی از شوبنده ها | جدول یاب,معنی از شوبنده ها,از شوبنده ها در معادل ابجد,jadvalyab.ir/search?q=%D8%A7%D8%B2+%D8%B4%D9%8...,jadvalyab.ir,158
1,تجارت و اقتصاد,بیت‌کوین کش یک ارز مجازی مشهور است و بیت‌کوین ...,عکس بیت‌کوین کش برای پروفایل عکس و والپیپرهای ...,عکس بیت‌کوین کش برای پروفایل,عکس بیت‌کوین کش برای پروفایل,عکس بیت کوین با کیفیت 4K عکس ارزهای دیجیتال عک...,jowhareh.com/photo/%D8%B9%DA%A9%D8%B3-%D8%A8%D...,jowhareh.com,3268
2,سلامت,نوبت دهی دکتر مهناز عابدینی متخصص رادیولوژی و ...,دکتر مهناز عابدینی متخصص رادیولوژی و سونوگرافی...,دکتر مهناز عابدینی متخصص رادیولوژی و سونوگرافی...,دکتر مهناز عابدینی,آدرس و تلفن دکتر مهناز عابدینی نظرات و تجربیات...,doctor-yab.ir/Search/14773/%D8%AF%DA%A9%D8%AA%...,doctor-yab.ir,175
3,تکنولوژی و کامپبوتر,نرم افزار Geph برای اندروید یک پلت‌فرم چندسکوی...,دانلود تحریم‌گذر Geph برای اندروید خانه/اندروی...,دانلود تحریم‌گذر Geph برای اندروید,دانلود تحریم‌گذر Geph برای اندروید,دانلود نرم افزار Geph,palexe.site/dl/geph-android/,palexe.site,3402
4,تکنولوژی و کامپبوتر,سری جدید تلویزیون‌های هوشمند سامسونگ که با نام...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ,راه‌اندازی تلویزیون همگام‌سازی کنترل اتصال به ...,rokhdadeghtesadi.ir/43874/,rokhdadeghtesadi.ir,3811


In [236]:
test = pd.read_csv('../data/yektanet_test.csv')
test.head()

,description,text_content,title,h1,h2,url,domain,id
0,پیام ناشناس یک ابزار رایگان برای ساخت چالش های...,پیام ناشناس / چالش دریافت پیام های ناشناس با ل...,پیام ناشناس / چالش دریافت پیام های ناشناس با ل...,NaN,پیام ناشناس / چالش دریافت پیام های ناشناس با ل...,abzarek.ir/service-p/msg/824757/,abzarek.ir,3291
1,اقتصادنیوز: سید امیر احمد علیه السلام ملقب به ...,حرم شاهچراغ شیراز در گذر زمان+تصاویر حرم شاهچر...,حرم شاهچراغ شیراز در گذر زمان+تصاویر,حرم شاهچراغ شیراز در گذر زمان+تصاویر,NaN,eghtesadnews.com/%D8%A8%D8%AE%D8%B4-%D8%A7%D8%...,eghtesadnews.com,5053
2,آخرین خبر های بین المللی و آخرین تحولات جهان، ...,آخرین خبر | زلنسکی پیام پوتین را دریافت کرد ✖...,آخرین خبر | زلنسکی پیام پوتین را دریافت کرد,زلنسکی پیام پوتین را دریافت کرد,NaN,akharinkhabar.ir/world/9332333/%D8%B2%D9%84%D9...,akharinkhabar.ir,564
3,آهنگ گاهی دریدن لازم نیست با صدای امیر تتلو با...,آهنگ گاهی دریدن لازم نیست امیر تتلو آهنگ گاهی ...,آهنگ گاهی دریدن لازم نیست امیر تتلو,آهنگ گاهی دریدن لازم نیست امیر تتلو,گاهی دریدن لازم نیست امیر تتلو,behmusic.com/music/%DA%AF%D8%A7%D9%87%DB%8C-%D...,behmusic.com,3607
4,کیهان با اشاره به حوادث و ناآرامیهای اخیر کشور...,انتقاد کیهان از امام جمعه اهل سنت زاهدان/ با ا...,انتقاد کیهان از امام جمعه اهل سنت زاهدان/ با ا...,انتقاد کیهان از امام جمعه اهل سنت زاهدان/ با ا...,برچسب‌ها نظر شما آخرین اخبار سیاسی ایران در بح...,khabaronline.ir/news/1688132/%D8%A7%D9%86%D8%A...,khabaronline.ir,2358


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش و مهندسی ویژگی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در هنگام کار با داده‌های متنی، پیش‌پردازش داده‌ها به کمک تکنیک‌های پردازش زبان طبیعی یکی از مراحل بسیار تاثیرگذار در یادگیری مدل و عملکرد نهایی است.
در تمرین «کامنت‌کاوی» فصل دسته‌بندی با چندین تکنیک رایج پیش‌پردازش خصوصاً برای زبان فارسی آشنا شدید. در این قسمت نیز می‌توانید تابعی بنویسید که یک رشته (<code>string</code>) را گرفته، اصلاحات موردنظر شما رو روی متن اعمال کرده و در نهایت نتیجه را با فرمت یک رشته (<code>string</code>) خروجی دهد. بررسی و تحلیل کلمات موجود در مجموعه‌داده از نظر تعداد رخداد نیز می‌تواند شما را در پیش‌پردازش بهتر یاری کند.
</font>
</p>


In [237]:
train.isna().sum()

category           0
description       46
text_content       0
title              0
h1               358
h2              1439
url                0
domain             5
id                 0
dtype: int64

In [238]:
test.isna().sum()

description       3
text_content      0
title             0
h1               33
h2              118
url               0
domain            1
id                0
dtype: int64

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
با توجه به وجود مقادیر NaN در ستون description بهتر است از ستون text_content یا از ستون title برای پیش‌بینی موضوع استفاده کنیم
</font>
</p>

In [239]:
x_train = pd.DataFrame({'text_content': train['text_content']})
x_test = pd.DataFrame({'text_content': test['text_content']})

y_train = pd.DataFrame({'category': train['category']})

In [240]:
x_train

,text_content
0,معنی از شوبنده ها | جدول یاب از شوبنده ها 381
1,عکس بیت‌کوین کش برای پروفایل عکس و والپیپرهای ...
2,دکتر مهناز عابدینی متخصص رادیولوژی و سونوگرافی...
3,دانلود تحریم‌گذر Geph برای اندروید خانه/اندروی...
4,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ...
...,...
4784,قرص ال دی با دوفاستون تداخل داره؟ پرسش (1397/0...
4785,فروش نقدی و اقساطی ماشین لباسشویی اسنوا سری ها...
4786,قهوه درویش در مشهد - پشت بام تهران ورود/ثبت نا...
4787,کمک غیرمستقیم فیفا به کی‌روش! | تمام رقیبان تی...


In [241]:
le = LabelEncoder()
y_train['category'] = le.fit_transform(y_train['category'])

In [242]:
y_train

,category
0,21
1,2
2,11
3,4
4,4
...,...
4784,11
4785,7
4786,13
4787,20


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در هنگام تعبیه‌سازی به کمک کتابخانه‌ی <code>scikit-learn</code>
از آنجا که ممکن است توکن‌ساز (Tokenizer) مورد استفاده در این کتابخانه مناسب زبان فارسی نباشد، بهتر است از توکن‌سازهای مناسب این زبان استفاده کنیم. کافیست تابعی بنویسید که یک رشته (<code>string</code>) را گرفته و به کمک کتابخانه‌ی موردنظر شما (مثل <code>word_tokenize</code> در کتابخانه‌ی هضم) توکن‌های آن را جدا کند. خروجی تابع لیستی از توکن‌ها خواهد بود.
</font>
</p>


In [243]:
def remove_emoji(text):
    return emoji.demojize(text, delimiters=("", ""))

In [244]:
punctuations = ['(', ')', '"', "'", '.', ',', '،', ':', ';',
                '؛', '!', '?', '»', '«', '{', '}', ']', '[', '', '-', '/', '|', '؟', '_', 'ـ', '-', '-', ',-', '--']

normalizer = hz.Normalizer()
lemmatizer = hz.Lemmatizer()

token = hz.WordTokenizer()


def preprocessing (text) :
    digitless_text = ''
    # delete digits and new lines cahracters
    for letter in text:
        if not letter.isdigit() and letter != '\n' :
            digitless_text += letter
    # add semi-space to text
    text = normalizer.normalize(digitless_text)
    # tokenize text
    text = token.tokenize(text)
    # remove punctuation
    filtered = [word for word in text if word not in punctuations]
    # lemmatize words
    filtered = [lemmatizer.lemmatize(word).split('#', 1)[0] for word in filtered]
    # remove stopwords
    filtered = [word for word in filter(None, filtered) if word not in hz.stopwords_list()]
    
    return filtered

In [246]:
def token_counter(texts):
    count_dict = {}

    for text in texts :
        tokens = preprocessing(text)
        for token in tokens :
            if token in count_dict :
                count_dict[token] += 1
            else :
                count_dict[token] = 1

    return count_dict

In [247]:
x_train['text_content'] = x_train['text_content'].apply(remove_emoji)
x_test['text_content'] = x_test['text_content'].apply(remove_emoji)

In [171]:
tokens_count = token_counter(x_train['text_content'])

sorted_tokens_count = sorted(tokens_count, key = lambda token: tokens_count[token], reverse = True)
sorted_tokens_count[:10]

['ایران',
 'دانلود',
 ',-',
 '--',
 'قیمت',
 'تهران',
 'سال',
 'آهنگ',
 'خودرو',
 'خرید']

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
توجه داشته باشید انتخاب این‌که از کدام ویژگی‌ها (متن‌ها) به‌عنوان ورودی الگوریتم استفاده کنید بر عهده‌ی خودتان است. می‌توانید تنها از یک ستون مثل <code>title</code> استفاده کنید یا می‌توانید به‌نحوی متن یا حتی بردار ویژگی هر ستون را با همدیگر ترکیب کنید. همچنین فراموش نکنید که ستون متغیر هدف یعنی <code>category</code> نیاز به کدگذاری دارد.
</font>
</p>


In [248]:
x_train['text_content'] = x_train['text_content'].apply(preprocessing)
x_test['text_content'] = x_test['text_content'].apply(preprocessing)

x_train['text_content'] = x_train['text_content'].apply(lambda x: ' '.join(x))
x_test['text_content'] = x_test['text_content'].apply(lambda x: ' '.join(x))

In [249]:
tfidf = TfidfVectorizer(ngram_range=(1, 1), max_features=5000)

tfidf.fit(x_train['text_content'])
encoded_text_train = tfidf.transform(x_train['text_content']).toarray()
encoded_text_test = tfidf.transform(x_test['text_content']).toarray()

In [250]:
encoded_text_train = pd.DataFrame(encoded_text_train)
encoded_text_train.columns = encoded_text_train.columns.astype(str)

encoded_text_test = pd.DataFrame(encoded_text_test)
encoded_text_test.columns = encoded_text_test.columns.astype(str)

In [251]:
x_train, x_val, y_train, y_val = train_test_split(encoded_text_train, y_train, test_size=0.2, random_state=13)

x_train.reset_index(drop=True, inplace=True)
x_val.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_val.reset_index(drop=True, inplace=True)

In [252]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.059355,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.026029,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.016267,0.0,0.0,0.0,0.0,0.0,0.0
3829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مدل‌سازی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
اکنون وقت آن رسیده که الگوریتم یادگیری ماشین موردنظر خود را بر روی داده‌های آموزشی اجرا کنید. در انتخاب الگوریتم کاملاً آزاد هستید. برای این بخش می‌توانید ابتدا هر ورودی متن را به کمک تکنیک‌های معرفی‌شده در درسنامه‌های این فصل (مثل Bag-of-Word یا Tf-idf) به بردارهای ویژگی عددی تبدیل کنید. سپس این بردارها را همراه با لیست برچسب‌های صحیح به الگوریتم بدهید تا مدل آموخته شود. علاوه بر این می‌توانید تمام این مراحل را در یک خط لوله‌ (<code>Pipeline</code>) نیز تعریف کنید.
</font>
</p>

In [255]:
model = XGBClassifier()
model.fit(x_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    معیاری که برای ارزیابی عملکرد مدل انتخاب کرده‌ایم، <code>F1-score</code> نام دارد و از رویکرد میانگین‌گیری وزن‌دار (<code dir=ltr>average='weighted'</code>) استفاده می‌شود.
    <br>
    پیشنهاد می‌شود با توجه به این معیار، عملکرد مدل خود را بر روی مجموعه‌ی آموزش یا اعتبارسنجی ارزیابی کنید و طبق نتایج به‌دست‌آمده پارامترهای مدل خود را بهتر تنظیم کنید.
</font>
</p>

In [256]:
y_train_pred = model.predict(x_train)
y_val_pred = model.predict(x_val)

print('Train F1 Score: ', f1_score(y_train, y_train_pred, average='weighted'))
print('Validation F1 Score: ', f1_score(y_val, y_val_pred, average='weighted'))

Train F1 Score:  0.9976519975134572
Validation F1 Score:  0.7788118731458247


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    پس از مهندسی ویژگی و مدل‌سازی، الگوریتمی دارید که می‌تواند شما را از متغیرهای مستقل به متغیر هدف برساند.
    <br>
    از این مدل برای پیش‌بینی نمونه‌های موجود در مجموعه‌ی آزمون استفاده کنید و نتایج را در یک دیتافریم تک‌ستونه با نام <code>submission</code> و در قالب زیر آماده کنید. توجه داشته باشید که ترتیب پیش‌بینی شما اهمیت دارد یعنی به عنوان مثال پیش‌بینی مدل برای نمونه‌ی آزمون <code>m</code> ام را باید در ردیف <code>m</code> ام این دیتافریم ذخیره کنید.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|<code>category</code>|پیش‌بینی مدل شما (از جنس رشته)|
    
</font>
</div>
</center>

In [257]:
submission = model.predict(encoded_text_test)
submission = pd.DataFrame({'category': submission})
submission['category'] = le.inverse_transform(submission['category'])
submission.head()

,category
0,هنر و سرگرمی
1,حقوق و دولت و سیاست
2,حقوق و دولت و سیاست
3,موسیقی
4,حقوق و دولت و سیاست


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد.
</font>
</p>

In [258]:
import zipfile
import joblib

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)
file_names = ['text_categorization.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['text_categorization.ipynb', 'submission.csv']


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
💭 اضافه: ابرِ کلمات (Word Cloud)
</font>
</h2>

<center>
<img src="wordcloud.png">
</center>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
یکی از کتابخانه‌های بسیار جالب مرتبط با متن در پایتون، <a href="https://github.com/amueller/word_cloud"><code>WordCloud</code></a> نام دارد. این کتابخانه به شما کمک می‌کند تا ابری از پرتکرارترین توکن‌های موجود در یک مجموعه‌متن را به شکلی زیبا به تصویر بکشید. خوشبختانه نسخه‌ی فارسی این کتابخانه نیز وجود دارد که می‌توانید از <a href="https://github.com/alihoseiny/word_cloud_fa">این لینک</a> به صفحه‌ی گیت‌هاب آن مراجعه کنید. حتی می‌توانید به‌صورت دلخواه تصویری را تعیین کنید تا نمایش نهایی توکن‌ها تداعی‌کننده‌ی تصویر موردنظر شما باشد.
</font>
</p>

In [ ]:
cloud_text = ''
for text in X_train:
    cloud_text += text + ' '

In [ ]:
from wordcloud_fa import WordCloudFa

wc = WordCloudFa(width=1200, height=800, persian_normalize=True, include_numbers=False, max_words=120, background_color='white', min_font_size=10, max_font_size=100)
word_cloud = wc.generate(cloud_text)
image = word_cloud.to_image()
image.show()
image.save('wordcloud.png')

<h4 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>راهنمایی</b>
</font>
</h4>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    ۱. از تکنیک n-gram کمک بگیرید.
    <br>
    ۲. توازن مجموعه‌داده را بررسی کنید.
    <br>
    ۳. در پیش‌پردازش خود می‌توانید حذف حروف اضافه و اعداد، حذف کلمات توقف، نرمال‌سازی و... را آزمایش کنید.
</font>
</p>